# API Research
- API Keys
- Imports
- Multi Region Search Call
- Single Estimate
- Batch Estimate
- Multi Region/Multi Query Search Call
- Export

API Keys

In [30]:
MY_API_KEY = "ZM85KK6A7M4CG1NA7PJYP6KZVDE9"
#MY_API_KEY = "7ME5B8JAG6M4KWMVVKZXQH9S8NCS"

Imports

In [3]:
import sys
import requests
import pprint
import json
import pandas as pd


Multi Region Search Call (First Iteration)
- Define regions with "regions_to_search" list
- Query variable is a free text string to which the model will try to match to the appropriate ID. Still refer to the Data Explore

In [4]:
def get_data_for_regions(regions):
    url = "https://beta4.api.climatiq.io/search"

    # Fill in the string with what you want the model to fuzzy match on for Activity ID
    query = "Electricity"

    query_params = {
        "query": query,
        "data_version": "^1",
        'results_per_page': 800, #helps with decreaseing process time per call
        #'region_fallback': True #This will not work for a list of regions as a FYI
        #'source_lca_activity': #Refer to Data Explore
        #'year':
    }

    authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

    region_data_list = []

    for region in regions:
        query_params["region"] = region
        search_response = requests.get(url, params=query_params, headers=authorization_headers).json()
        search_results = search_response.get("results", [])
        region_data_df = pd.json_normalize(search_results)
        region_data_list.append(region_data_df)

    return pd.concat(region_data_list, ignore_index=True)

#List of Regions to Search. Use regions specified in the Data Explore
regions_to_search = ["US","US*", "*CA","CA*", "AT*", "Africa", "GB", "CN","EURASIA", "GLOBAL", "AU", "MX", "JP", "CN"]

# Get data for the specified regions
combined_df = get_data_for_regions(regions_to_search)

# Printing the combined DataFrame
print("Combined Data for all regions:")

print(combined_df.shape)
combined_df

Combined Data for all regions:
(615, 28)


,activity_id,id,name,category,sector,source,source_link,source_dataset,uncertainty,year,...,access_type,supported_calculation_methods,factor,factor_calculation_method,factor_calculation_origin,constituent_gases.co2e_total,constituent_gases.co2e_other,constituent_gases.co2,constituent_gases.ch4,constituent_gases.n2o
0,electricity-supply_grid-source_biomass_waste,418c5843-83d4-4d67-a1b8-bd5662f6243c,Electricity generated from biomass and waste,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",3.8826,ar5,source,3.8826,NaN,NaN,NaN,NaN
1,electricity-supply_grid-source_coal,8e69d067-fbcb-464d-9ac9-31f06928cd60,Electricity generated from coal,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",13.3786,ar5,source,13.3786,NaN,NaN,NaN,NaN
2,electricity-supply_grid-source_gas,1ed4d0f2-5861-4ed0-b058-d68f4d84593d,Electricity generated from gas,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",12.0212,ar5,source,12.0212,NaN,NaN,NaN,NaN
3,electricity-supply_grid-source_hydro,bcb280a2-71e4-483c-a663-cc4049327e66,Electricity generated from hydro,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",0.1258,ar5,source,0.1258,NaN,NaN,NaN,NaN
4,electricity-supply_grid-source_nuclear,3a69abb3-d831-47a1-af81-20a3995eb19a,Electricity generated from nuclear,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",0.0594,ar5,source,0.0594,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,electricity-supply_grid-source_supplier_mix,de2c271c-8f54-4583-96b0-39282e413766,Electricity supplied from grid,Electricity,Energy,CT,https://www.climate-transparency.org/g20-clima...,Brown to Green Report,NaN,2018,...,public,[ar4],0.5550,ar4,source,0.5550,None,0.555,None,None
611,electricity-supply_grid-source_supplier_mix,eb0db4f5-d1c4-46eb-ac2b-cc8a9dcdcbf0,Electricity supplied from grid,Electricity,Energy,CT,https://www.climate-transparency.org/g20-clima...,Climate Transparency Report,NaN,2019,...,public,[ar4],0.5560,ar4,source,0.5560,None,0.556,None,None
612,electricity-supply_grid-source_supplier_mix,4605d4f1-5095-4f3e-99aa-4a136813bc57,Electricity supplied from grid,Electricity,Energy,CT,https://www.climate-transparency.org/g20-clima...,Climate Transparency Report,NaN,2020,...,public,[ar4],0.5374,ar4,source,0.5374,None,0.5374,None,None
613,electricity-supply_grid-source_supplier_mix,19f28a64-a761-4b68-a6b8-65977c67c4bf,Electricity supplied from grid,Electricity,Energy,CT,https://www.climate-transparency.org/g20-clima...,Climate Transparency Report,NaN,2021,...,public,[ar4],0.5572,ar4,source,0.5572,None,0.5572,None,None


Single Estimates
- Not as effective as batching or looping through multiple but might be useful from an RR perspective

In [37]:
# activity_id = search_response["results"][0]["activity_id"]
activity_id = 'consumer_services-type_all_other_food_drinking_places'
region = 'US'

parameters = {
        "money": 500,
        "money_unit": "usd"
    }

json_body = {
    "emission_factor": {
        "activity_id": activity_id,
        "region": region,
        "data_version": "3.3"
        # More filters are possible here. See the full documentation for more options
    },
    # Specify how much energy we're estimating for
    "parameters": parameters
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the estimate endpoint with a json body and the correct authorization headers
response = requests.post("https://beta4.api.climatiq.io/estimate", json=json_body, headers=authorization_headers)
# The response is also json.
pprint.pprint(response.json())

{'activity_data': {'activity_unit': 'usd', 'activity_value': 500.0},
 'audit_trail': 'selector',
 'co2e': 85.49999999999999,
 'co2e_calculation_method': 'ar4',
 'co2e_calculation_origin': 'climatiq',
 'co2e_unit': 'kg',
 'constituent_gases': {'ch4': 0.5,
                       'co2': 71.5,
                       'co2e_other': 1.5,
                       'co2e_total': None,
                       'n2o': 0.0},
 'emission_factor': {'access_type': 'public',
                     'activity_id': 'consumer_services-type_all_other_food_drinking_places',
                     'category': 'Food and Beverage Services',
                     'data_quality_flags': [],
                     'id': 'a06624f4-9efd-42f9-b514-44c15883ca7e',
                     'name': 'All other food and drinking places',
                     'region': 'US',
                     'source': 'EPA',
                     'source_dataset': 'Supply Chain Factors Dataset '
                                       '(commodities)',
   

Batch Estimate

parameters_to_use= {
    ##Area
    'area':,
    'area_unit':,
    ##Area over time
    'area':,
    'area_unit':,
    'time':,
    'time_unit':,
    ##container over distance
    'twenty_foot_equivalent':,
    'distance':,
    'distance_unit':,
    ##Data
    'data':,
    'data_unit':,
    ##DataOverTime
    'data':,
    'data_unit':,
    'time':,
    'time_unit':,
    ##Distance
    'distance':,
    'distace_unit':,
    ##Distance over time
    'distance':,
    'distance _unit':,
    'time':,
    'time_unit':,
    ##Energy
    'energy':,
    'energy_unit':,
    ##Money
    'money':,
    'money_unit':,
    ##Number
    'number':,
    ##NumberOverTime
    'number':,
    'time':,
    'time_unit':,
    ##PassengerOverDistance
    'passengers':,
    'distance':,
    'distance_unit':,
    ##Time
    'time':,
    'time_unit':,
    ##Volume
    'volume':,
    'volume_unit':,
    ##Weight
    'weight':,
    'weight_unit':,
    ##WeightOverDistance
    'weight':,
    'weight_unit':,
    'distance':,
    'distance_unit':,
    ##WeightOverTime
    'weight':,
    'weight_unit':,
    'time':,
    'time_unit':

}

In [ ]:
activity_id = 'electricity-supply_grid-source_supplier_mix'

region = 'US*'

def api_runner():

    MY_API_KEY = "ZM85KK6A7M4CG1NA7PJYP6KZVDE9"
    
    parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

    # Notice that the JSON body here is an array, with each entry being an estimate
    json_body = [
    {
        "emission_factor": {
            # Using the same activity ID and region as before
            "activity_id": activity_id,
            "region": region,
            "data_version": "^1"
        },
        "parameters": {
            "energy": 10,
            "energy_unit": "kWh"
        }
    },
    {
        "emission_factor": {
            # Using the same activity ID and region as before
            "activity_id": activity_id,
            "region": region,
            "data_version": "^1"
        },
        "parameters": {
            "energy": 100,
            "energy_unit": "kWh"
        }
    },
    {
        "emission_factor": {
            # Using the same activity ID and region as before
            "activity_id": activity_id,
            "region": region,
            "data_version": "^1"
        },
        "parameters": {
            "energy": 600,
            "energy_unit": "kWh"
        }
    },
    {
        "emission_factor": {
            # Using the same activity ID and region as before
            "activity_id": activity_id,
            "region": region,
            "data_version": "^1"
        },
        "parameters": {
            "energy": 10000,
            "energy_unit": "kWh"
        }   
    },
    {
        "emission_factor": {
            # Using the same activity ID and region as before
            "activity_id": activity_id,
            "region": region,
            "data_version": "^1"
        },
        "parameters": {
            "energy": 400,
            "energy_unit": "kWh"
        }
    }
    ]

    # You must always specify your AUTH token in the "Authorization" header like this.
    authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

    # We send a POST request to the /batch endpoint with a json body and the correct authorization headers
    response = requests.post("https://beta4.api.climatiq.io/batch", json=json_body, headers=authorization_headers)
    # The response is also json.
    response_json = response.json()
    return response_json

pd.set_option('display.max_columns', None)
df = pd.json_normalize(api_runner()['results'])
df

,co2e,co2e_unit,co2e_calculation_method,co2e_calculation_origin,audit_trail,emission_factor.name,emission_factor.activity_id,emission_factor.id,emission_factor.access_type,emission_factor.source,emission_factor.source_dataset,emission_factor.year,emission_factor.region,emission_factor.category,emission_factor.source_lca_activity,emission_factor.data_quality_flags,constituent_gases.co2e_total,constituent_gases.co2e_other,constituent_gases.co2,constituent_gases.ch4,constituent_gases.n2o,activity_data.activity_value,activity_data.activity_unit
0,9.02706,kg,ar4,source,selector,Electricity supplied from grid,electricity-supply_grid-source_supplier_mix,ff8bd67a-9786-4908-841e-b7224426c1f0,public,EPA,eGRID,2022,US-WY,Electricity,electricity_generation,[],9.02706,None,8.96137,0.00096,0.00014,10.0,kWh
1,90.27060,kg,ar4,source,selector,Electricity supplied from grid,electricity-supply_grid-source_supplier_mix,ff8bd67a-9786-4908-841e-b7224426c1f0,public,EPA,eGRID,2022,US-WY,Electricity,electricity_generation,[],90.27060,None,89.61370,0.00960,0.00140,100.0,kWh
2,541.62360,kg,ar4,source,selector,Electricity supplied from grid,electricity-supply_grid-source_supplier_mix,ff8bd67a-9786-4908-841e-b7224426c1f0,public,EPA,eGRID,2022,US-WY,Electricity,electricity_generation,[],541.62360,None,537.68220,0.05760,0.00840,600.0,kWh
3,9027.06000,kg,ar4,source,selector,Electricity supplied from grid,electricity-supply_grid-source_supplier_mix,ff8bd67a-9786-4908-841e-b7224426c1f0,public,EPA,eGRID,2022,US-WY,Electricity,electricity_generation,[],9027.06000,None,8961.37000,0.96000,0.14000,10000.0,kWh
4,361.08240,kg,ar4,source,selector,Electricity supplied from grid,electricity-supply_grid-source_supplier_mix,ff8bd67a-9786-4908-841e-b7224426c1f0,public,EPA,eGRID,2022,US-WY,Electricity,electricity_generation,[],361.08240,None,358.45480,0.03840,0.00560,400.0,kWh


Multi Region/Multi Query Search Call

In [12]:
import pandas as pd
import requests

def get_data_for_regions(regions, queries):
    url = "https://beta4.api.climatiq.io/search"

    query_params = {
        "data_version": "^1",
        'results_per_page': 800,
    }

    authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

    region_data_list = []

    for region in regions:
        for query in queries:
            query_params["region"] = region
            query_params["query"] = query
            search_response = requests.get(url, params=query_params, headers=authorization_headers).json()
            search_results = search_response.get("results", [])
            region_data_df = pd.json_normalize(search_results)
            region_data_list.append(region_data_df)

    return pd.concat(region_data_list, ignore_index=True)

# List of Regions to Search. Use regions specified in the Data Explore
regions_to_search = ["US","US*", "*CA","CA*", "AT*", "Africa", "GB", "CN","EURASIA", "GLOBAL", "AU", "MX", "JP"]

# List of Queries to loop through. Refer to Data Explore
queries_to_search = ["Electricity", "Transport", "Consumer Goods and Services", "Organizational Activities"]

# Get data for the specified regions and queries
combined_df_2 = get_data_for_regions(regions_to_search, queries_to_search)

# Printing the combined DataFrame
print("Combined Data for all regions and queries:")
combined_df_2.shape
combined_df_2


Combined Data for all regions and queries:


,activity_id,id,name,category,sector,source,source_link,source_dataset,uncertainty,year,...,access_type,supported_calculation_methods,factor,factor_calculation_method,factor_calculation_origin,constituent_gases.co2e_total,constituent_gases.co2e_other,constituent_gases.co2,constituent_gases.ch4,constituent_gases.n2o
0,electricity-supply_grid-source_biomass_waste,418c5843-83d4-4d67-a1b8-bd5662f6243c,Electricity generated from biomass and waste,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",3.8826,ar5,source,3.8826,NaN,NaN,NaN,NaN
1,electricity-supply_grid-source_coal,8e69d067-fbcb-464d-9ac9-31f06928cd60,Electricity generated from coal,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",13.3786,ar5,source,13.3786,NaN,NaN,NaN,NaN
2,electricity-supply_grid-source_gas,1ed4d0f2-5861-4ed0-b058-d68f4d84593d,Electricity generated from gas,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",12.0212,ar5,source,12.0212,NaN,NaN,NaN,NaN
3,electricity-supply_grid-source_hydro,bcb280a2-71e4-483c-a663-cc4049327e66,Electricity generated from hydro,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",0.1258,ar5,source,0.1258,NaN,NaN,NaN,NaN
4,electricity-supply_grid-source_nuclear,3a69abb3-d831-47a1-af81-20a3995eb19a,Electricity generated from nuclear,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"[ar4, ar5]",0.0594,ar5,source,0.0594,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,organizational_activity-type_public_administra...,65847934-ad66-466c-a6e3-8762930ad346,Public administration and defense services/com...,Government Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,None,2019,...,public,"[ar4, ar5]",0.1942,ar5,source,0.1942,None,None,None,None
1803,professional_services-type_computer_related_se...,4e5c4f65-eb1c-43e3-b112-12f69d2841cc,Computer and related services,Professional Services and Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,None,2019,...,public,"[ar4, ar5]",0.1263,ar5,source,0.1263,None,None,None,None
1804,professional_services-type_foundry_work_services,d877f3ff-3637-405a-8a5e-1ff687d17060,Foundry work services,Professional Services and Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,None,2019,...,public,"[ar4, ar5]",1.2368,ar5,source,1.2368,None,None,None,None
1805,professional_services-type_other_business_serv...,96793574-4e26-460a-9692-ebe0ba5edf30,Other business services,Professional Services and Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,None,2019,...,public,"[ar4, ar5]",0.1027,ar5,source,0.1027,None,None,None,None


In [21]:
combined_df_2.to_csv(r'C:\Users\jackg\OneDrive - University of Tennessee\Desktop\COSC Master File\COSCfiles\API Code\API Calls\combined_df_2.csv', index=False)

In [22]:
df_3 = pd.read_csv(r"C:\Users\jackg\OneDrive - University of Tennessee\Desktop\COSC Master File\COSCfiles\API Code\API Calls\combined_df_2.csv")
df_3

,activity_id,id,name,category,sector,source,source_link,source_dataset,uncertainty,year,...,access_type,supported_calculation_methods,factor,factor_calculation_method,factor_calculation_origin,constituent_gases.co2e_total,constituent_gases.co2e_other,constituent_gases.co2,constituent_gases.ch4,constituent_gases.n2o
0,electricity-supply_grid-source_biomass_waste,418c5843-83d4-4d67-a1b8-bd5662f6243c,Electricity generated from biomass and waste,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",3.8826,ar5,source,3.8826,NaN,NaN,NaN,NaN
1,electricity-supply_grid-source_coal,8e69d067-fbcb-464d-9ac9-31f06928cd60,Electricity generated from coal,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",13.3786,ar5,source,13.3786,NaN,NaN,NaN,NaN
2,electricity-supply_grid-source_gas,1ed4d0f2-5861-4ed0-b058-d68f4d84593d,Electricity generated from gas,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",12.0212,ar5,source,12.0212,NaN,NaN,NaN,NaN
3,electricity-supply_grid-source_hydro,bcb280a2-71e4-483c-a663-cc4049327e66,Electricity generated from hydro,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",0.1258,ar5,source,0.1258,NaN,NaN,NaN,NaN
4,electricity-supply_grid-source_nuclear,3a69abb3-d831-47a1-af81-20a3995eb19a,Electricity generated from nuclear,Electricity,Energy,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",0.0594,ar5,source,0.0594,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,organizational_activity-type_public_administra...,65847934-ad66-466c-a6e3-8762930ad346,Public administration and defense services/com...,Government Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",0.1942,ar5,source,0.1942,NaN,NaN,NaN,NaN
1803,professional_services-type_computer_related_se...,4e5c4f65-eb1c-43e3-b112-12f69d2841cc,Computer and related services,Professional Services and Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",0.1263,ar5,source,0.1263,NaN,NaN,NaN,NaN
1804,professional_services-type_foundry_work_services,d877f3ff-3637-405a-8a5e-1ff687d17060,Foundry work services,Professional Services and Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",1.2368,ar5,source,1.2368,NaN,NaN,NaN,NaN
1805,professional_services-type_other_business_serv...,96793574-4e26-460a-9692-ebe0ba5edf30,Other business services,Professional Services and Activities,Organizational Activities,EXIOBASE,https://zenodo.org/record/5589597#.Yh9_Zi8w1ao,EXIOBASE 3,NaN,2019,...,public,"['ar4', 'ar5']",0.1027,ar5,source,0.1027,NaN,NaN,NaN,NaN
